In [4]:
import requests
import json
import base64
import configparser
import pandas as pd
import csv
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display

# Load configuration
config = configparser.ConfigParser()
config.read("config/config.properties")

# Define widgets for user input
jira_url_widget = widgets.Text(value=config['jira']['url'], description='Jira URL:', disabled=False)
okta_domain_widget = widgets.Text(value=config['okta']['domain'], description='Okta Domain:', disabled=False)
okta_api_token_widget = widgets.Password(description='Okta API Token:', disabled=False)


# Define test cases
TEST_CASES = [
    {"test_name": "Non-admin user", "ad_group": True, "jira_access": True, "okta_access": True, "expected": "Access as non-admin user"},
    {"test_name": "Admin user", "ad_group": True, "jira_access": True, "okta_access": True, "expected": "Access as admin user"},
    {"test_name": "No AD", "ad_group": False, "jira_access": True, "okta_access": True, "expected": "Access to privileged site denied"},
    {"test_name": "Local Account", "ad_group": False, "jira_access": None, "okta_access": False, "expected": "Account Creation fails"},
    {"test_name": "Okta Local Account", "ad_group": False, "jira_access": None, "okta_access": True, "expected": "Group Assignment should fail for Product, Site Access"},
]

# Dropdown to select individual test case
test_case_names = [test["test_name"] for test in TEST_CASES]
test_case_dropdown = widgets.Dropdown(options=test_case_names, description="Test Case:")
execute_button = widgets.Button(description="Run Selected Test")

display(jira_url_widget, okta_domain_widget, okta_api_token_widget, test_case_dropdown, execute_button)

# Function to authenticate via Jira
def authenticate_jira(username, api_token):
    auth_header = 'Basic ' + base64.b64encode(f"{username}:{api_token}".encode()).decode()
    headers = {
        "Authorization": auth_header,
        "Accept": "application/json"
    }
    response = requests.get(f"{jira_url_widget.value}/rest/api/3/myself", headers=headers)
    return response

# Function to check if Okta user has Jira access
def check_okta_jira_access(username):
    url = f"{jira_url_widget.value}/rest/api/3/user?username={username}"
    headers = {
        "Authorization": f"Bearer {okta_api_token_widget.value}",
        "Accept": "application/json"
    }
    response = requests.get(url, headers=headers)
    return response

# Function to get user directories
def get_user_directories():
    url = f"{jira_url_widget.value}/rest/api/3/groups/picker"
    headers = {
        "Authorization": f"Bearer {okta_api_token_widget.value}",
        "Accept": "application/json"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    return {}

# Function to run a selected test case
def run_selected_test(button):
    selected_test_name = test_case_dropdown.value
    test = next((t for t in TEST_CASES if t["test_name"] == selected_test_name), None)
    if not test:
        print("Test case not found.")
        return
    
    username = config['users'].get(test['test_name'], None)
    jira_token = config['users'].get(f"{test['test_name']}_token", None)
    
    jira_status, okta_status = "Not Tested", "Not Tested"
    
    if test['jira_access']:
        response = authenticate_jira(username, jira_token)
        jira_status = "PASS" if response.status_code == 200 else "FAIL"
    
    if test['okta_access']:
        response = check_okta_jira_access(username)
        okta_status = "PASS" if response.status_code == 200 else "FAIL"
    
    result = {
        "Test Case": test['test_name'],
        "AD Group": "Yes" if test['ad_group'] else "No",
        "Jira Access": jira_status,
        "Okta Access": okta_status,
        "Expected Outcome": test['expected'],
        "Timestamp": datetime.utcnow().isoformat()
    }
    
    df = pd.DataFrame([result])
    df.to_csv(f"auth_audit_{datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')}.csv", index=False, mode='a', header=False)
    directory_data = get_user_directories()
    directory_df = pd.DataFrame(directory_data.get('groups', []))
    
    display(df)
    display(directory_df)

# Bind the function to button click
execute_button.on_click(run_selected_test)


Text(value='https://a9data.atlassian.net', description='Jira URL:')

Text(value='trial-1912226.okta.com', description='Okta Domain:')

Password(description='Okta API Token:')

Dropdown(description='Test Case:', options=('Non-admin user', 'Admin user', 'No AD', 'Local Account', 'Okta Lo…

Button(description='Run Selected Test', style=ButtonStyle())

/var/folders/0n/kvzzcvpx3fb9pgtj5n8tdvf00000gn/T/ipykernel_18044/1307762762.py:96: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "Timestamp": datetime.utcnow().isoformat()
/var/folders/0n/kvzzcvpx3fb9pgtj5n8tdvf00000gn/T/ipykernel_18044/1307762762.py:100: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df.to_csv(f"auth_audit_{datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')}.csv", index=False, mode='a', header=False)


,Test Case,AD Group,Jira Access,Okta Access,Expected Outcome,Timestamp
0,Non-admin user,Yes,FAIL,FAIL,Access as non-admin user,2025-02-10T20:06:12.672826


""


# Jira Cloud Authentication & Access Control Test Notebook

## Purpose
This Jupyter Notebook is designed to automate security and access control tests for Jira Cloud integrated with Okta authentication. It helps validate:

1. Whether non-admin users can access Jira Cloud.
2. If Active Directory (AD) group membership correctly enforces expected access levels.
3. Whether Okta-based authentication correctly enforces user permissions.
4. If unauthorized users can escalate privileges or gain access improperly.

## Features
- Uses **ipywidgets** for interactive execution.
- Runs authentication checks via **Jira REST API**.
- Validates whether Okta users have proper access to Jira Cloud.
- Generates an audit report in **CSV format**.
- Displays **user directory details** for further verification.
- Each test case can be executed independently using dropdown selection.

---